In [3]:
import sys
import os
sys.path.append("..")
import numpy as np
from sklearn.model_selection import cross_validate
from utils import invert_dict
from Modules.LoadData.load_data_ADNI import get_dataloader
BaselineSet = sorted(os.listdir('../Baselines/'))
TargetBaseline = BaselineSet[0]
print(BaselineSet)
print(TargetBaseline)
TargetDir = "../Baselines/" + TargetBaseline + "/ECNs_results/"
switcher = {
    "CN": 1,
    "LMCI": 2,
    "EMCI": 3,
    "MCI": 4,
    "SMC": 5
}
reverse_switcher = invert_dict(switcher)

['18-NOTEARS', '18-NRI', '19-DAGGNN']
18-NOTEARS


In [2]:
from sklearn import svm
import pandas as pd
model = svm.SVC(kernel='linear')

rows, cols = 6, 6
matrix = [[" "] * cols for _ in range(rows)]

for i in range(1,6):
    matrix[i][0] = reverse_switcher[i]
    matrix[0][i] = reverse_switcher[i]
    for j in range(1,6):
        if i>j:
            CausalityList = []
            LabelList = []
            groups = [i,j]
            dataloader = get_dataloader(batch_size = 1,parent=2)
            for data, id, group in dataloader:
                if switcher[group[0]] in groups:
                    file = TargetDir + id[0] + "_" + group[0] + ".npy"
                    causality = np.load(file)
                    CausalityList.append(causality)
                    LabelList.append(switcher[group[0]])

            CausalityAll = np.array(CausalityList)
            LabelAll = np.array(LabelList)
            CausalityAll_flatten = CausalityAll.reshape((CausalityAll.shape[0],-1))
            cv_results = cross_validate(model, CausalityAll_flatten, LabelAll, cv=5, return_train_score=True)
            # print(cv_results)
            # print("Baseline: ", TargetBaseline)
            # print("Groups: ", {reverse_switcher[group] for group in groups})
            # print("Test Scores: ", cv_results["test_score"], "\n-- Mean:", np.mean(cv_results["test_score"]), "  Var:", np.std(cv_results["test_score"]))
            # print("-- 95% confidence interval:", np.mean(cv_results["test_score"]),"±", 1.96*np.std(cv_results["test_score"]))
            matrix[i][j] = str("{:.4f}".format(np.mean(cv_results["test_score"])))+str("±")+ str("{:.4f}".format(1.96*np.std(cv_results["test_score"])))

df = pd.DataFrame(matrix)
df.to_excel("../Baselines/" + TargetBaseline + '/SVM_5cv_output.xlsx', index=False, header=False)

KeyboardInterrupt: 